In [4]:
# Import modules

In [404]:
import requests
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [458]:
# define parameters
raw_data = "twitter_raw_data.csv"
processed_data = "twitter_processed_data.csv"

In [ ]:
# Overview of data I have tested/experimented with

In [ ]:
Tweet objects:
    text - (tweet text)
    attachments - (media_key to join media object)
    author_id - (author_id to join user object)
    context_annotations - (domain and entity pairs, (with id, name and description) - hard to use for project (text based analytics required))
    created_at - tweet created datetime
    public_metrics - (RT, reply, like, quote counts)
    conversation_id - (key for..)
    entities - (info on position and value of: url, hashtags, symbols, user_mentions, polls, media in tweet)
    geo - (cant really use with essential access)
    id - (tweet id)
    in_reply_to_user_id - (id of user tweet is in reply to)
    lang - (language)
    referenced_tweets - (type,id of original tweet -e.g. if a retweet)
    reply_settings - (who can reply)
    source - (how accessed twitter, e.g. twitter for android)
    withheld - (withheld information, not very helpful)
    others - (not available or not useful with essential access)

User objects:
    id - (user id)
    created_at - (datetime profile created)
    description - (profile bio text)
    entities - (info on position and value of: url, hashtags, symbols, user_mentions, media in description bio)
    location - (place user created profile) frequently null or wrong
    pinned_tweet_id - (tweet id of pinned tweet)
    profile_image_url - (url of profile pic)
    protected - (if user is private or not - should always be False for me (No following))
    public_metrics - (followers_count, following_count, tweet_count, listed_count)
    url - (url in bio)
    username - (username)
    verified - (is user if verified as that person or not (think this is for big public figures only))
    withheld - (withheld information) 
    
Media objects:
    type - (type of media e.g. video/pic)
    height
    width
    duration_ms - (ms duration, only for video)
    public_metrics - (view count, only for video)
    preview_image_url - (image url)
    #the below three need user authentication
    non_public_metrics - (playback times/quartiled)
    organic_metrics - (playback times/quartiled tracked on organic basis)
    promoted_metrics - (playback times/quartiled on promoted basis)
    
Space objects: (live streaming objects, not available after ended) - not useful for this project
Poll objects: not useful for this project
Place objects: (most of the other info to do with place is unavailable/hard to use with essential access)

In [233]:
# Get all accessible data deemed appropriate
APP_ID = "22591708"
App_name = "alexanderfarqu2"
API_key = "ON4SJ0U3Sb0ODjUk6ZquERxiv"
API_key_secret = "g5JoHVL4uxCoFXLvEYTSojaEBPODyYmiI0bMeuvHYQt5JsyVE6"
bearer_token = "AAAAAAAAAAAAAAAAAAAAANy4WAEAAAAAlTukwlSz2gZnDi9xKeo3lp471qM%3DxU6od5cFrnKdcLHZHXx2I6cp99WT6QnKOpirCWfm75U42yXth2"
access_token = "1463912691429527556-rZ1rFU6GgR3a4xYcWdTbvsomCn5shl"
access_token_secret = "qqU34qYcJ0B8iEZC82DZ6OZqzrYy8IBJ51UGNsLdXvKZh"

client = tweepy.Client(bearer_token = bearer_token)

In [234]:
# query data (through trial and error of testing what tweets I want)
'''
querying notes:
    default is and logic (otherwise put or between words)
    some useful filters: is:, has:, lang: (not all available with essential access like place_country:)
        e.g. is:retweet lang:en has:images
    - negates anything:
        e.g. -snow -is:nullcast
    group and/or logic with brackets:
        e.g. (a b) or (c has:images)
'''

query = 'climbing has:media'

In [235]:
# list chosen from above
tweet_fields=['attachments', 'author_id', 'created_at', 'public_metrics', 'conversation_id', 'entities', 'geo', 'id', 'in_reply_to_user_id','lang', 'referenced_tweets','reply_settings','source']

In [236]:
# many other expansions: author_id,referenced_tweets.id,referenced_tweets.id.author_id,entities.mentions.username,attachments.poll_ids,attachments.media_keys,in_reply_to_user_id,geo.place_id
# just using expansions to get media and author info
expansions = ['author_id', 'attachments.media_keys']

In [237]:
# list chosen from above
user_fields = ['created_at','entities','id','profile_image_url','public_metrics','username','verified']

In [238]:
media_fields = ['height','width','public_metrics']

In [273]:
# extract data (not flattening as that would loose expansions data)
tweets = []
expan = []
n=0
for tweet in tweepy.Paginator(client.search_recent_tweets,
                          query=query, 
                          tweet_fields = tweet_fields, 
                          user_fields = user_fields, 
                          max_results = 10, 
                          expansions = expansions,
                          limit=2):
    tweets.append(tweet[0])
    expan.append(tweet[1])
    n+=1

In [274]:
# 1 dimensionalise tweets
tweets = [tweet for sub in tweets for tweet in sub]

In [318]:
# extract user information
users = []
for el in expan:
    users.append(el['users'])
    
# flatten
users = [user for sub in users for user in sub]    

In [335]:
# extract media information
medias = []
for el in expan:
    medias.append(el['media'])
    
# flatten
medias = [media for sub in medias for media in sub]    

In [448]:
# create tweet_df
tweet_df = pd.DataFrame()
# keys
tweet_df['author_id']= [tweet.author_id for tweet in tweets]
tweet_df['tweet_id']= [tweet.id for tweet in tweets]
tweet_df['attachments']= [tweet.attachments for tweet in tweets]
tweet_df['in_reply_to_user_id']= [tweet.in_reply_to_user_id for tweet in tweets] #drop?
tweet_df['referenced_tweets']= [tweet.referenced_tweets for tweet in tweets] #drop?

# cat variables
tweet_df['text']= [tweet.text for tweet in tweets]
tweet_df['lang']= [tweet.lang for tweet in tweets]
tweet_df['source']= [tweet.source for tweet in tweets]
tweet_df['reply_settings']= [tweet.reply_settings for tweet in tweets] # drop?
tweet_df['entities']= [tweet.entities for tweet in tweets]


# num variables
tweet_df['ret_count']= [tweet.public_metrics['retweet_count'] for tweet in tweets]
tweet_df['reply_count']= [tweet.public_metrics['reply_count']for tweet in tweets]
tweet_df['like_count']= [tweet.public_metrics['like_count']for tweet in tweets]
tweet_df['quote_count']= [tweet.public_metrics['quote_count']for tweet in tweets]

#date variables
tweet_df['created_at']= [tweet.created_at for tweet in tweets]

In [449]:
# get first media_key only from attachments
x = {'media_keys':[0]}
tweet_df['attachments'] = np.where(~tweet_df['attachments'].isna(),tweet_df['attachments'],x)
tweet_df['attachments'].isna().sum()

0

In [450]:
media_keys = []
for el in tweet_df['attachments']:
    for v in el.values():
        media_keys.append(v[0])

In [451]:
tweet_df['media_key'] = media_keys
tweet_df.drop(['attachments'], inplace = True, axis = 1)

In [453]:
# create users_df
users_df = pd.DataFrame()
# keys
users_df['author_id']= [user.id for user in users]

# num vars
users_df['profile_created_at']= [user.created_at for user in users]
users_df['followers_count']= [user.public_metrics['followers_count'] for user in users]
users_df['following_count']= [user.public_metrics['following_count'] for user in users]
users_df['user_tweet_count']= [user.public_metrics['tweet_count'] for user in users]

# cat vars
users_df['profile_entities']= [user.entities for user in users]
users_df['username']= [user.username for user in users]
users_df['verified']= [user.verified for user in users]

In [454]:
# media_fields = ['height','width','public_metrics']
media_df = pd.DataFrame()
# keys
media_df['media_key']= [media.media_key for media in medias]

# cat vars
media_df['media_type']= [media.type for media in medias]

In [455]:
# join data frames
master_df = tweet_df.merge(right = users_df, how = 'left', on = 'author_id').merge(right = media_df, how = 'left', on = 'media_key')

In [462]:
# save raw dataframe
master_df.to_csv(raw_data, index = False)

In [ ]:
cols_to_drop = ['reply_settings',]

In [ ]:
# qu.: 
#does hashtag in profile have a tendency for hashtag in recent tweets - link to profile created date?
#Do hashtags promote visibility of the tweet?
    (We are looking at only public tweets might have an affect on hashtags, and might be other correlations, people that use hashtags usually have moer followers)
    For all types of media?

In [ ]:
# metrics:
#     if referenced a tweet
#     if in reply to a tweet
#     language
#     source
#     entities - if hashtag
#     RT, replies, likes, quotes count
#     when_created
#     when profile_created
#     profile_entities - if hashtag
#     followers, following, no. tweets count
#     verified
#     media type (if any)

In [495]:
# Combine keys for ease
master_df['tweet_id'] = master_df['tweet_id'].astype(str)
master_df['author_id'] = master_df['author_id'].astype(str)
master_df['media_key'] = master_df['media_key'].astype(str)
master_df['id'] = master_df['tweet_id'] + "-" + master_df['author_id']  + "-" + master_df['media_key']
keys = ['media_key', 'author_id', 'tweet_id']
master_df.drop(keys, inplace = True, axis = 1)

In [ ]:
# Get list of cols - which ones to drop
master_df.columns

In [498]:
# Drop cols that will not be helpful
cols_to_drop = ['in_reply_to_user_id','referenced_tweets','lang','username']
master_df.drop(cols_to_drop, axis = 1, inplace = True)

In [535]:
master_df.head()

,text,source,reply_settings,ret_count,reply_count,like_count,quote_count,created_at,profile_created_at,followers_count,...,user_tweet_count,verified,type,id,profile_hashtag,profile_ht,tweet_ht,created_at_t,profile_created_at_t,time_diff
0,RT @brittany_sorg: Here is your #NewYears upda...,Twitter Web App,everyone,9,0,0,0,2022-01-03,2021-05-08,256,...,5598,False,NaN,1478018848087592963-1391116725849137154-0,0,0,1,2022-01-03,2021-05-08,240 days
1,"@NexoFinance #hokk is climbing the moon, do no...",Twitter for Android,everyone,0,0,0,0,2022-01-03,2021-11-05,175,...,4764,False,photo,1478018765589729285-1456752050486841345-3_1478...,0,0,1,2022-01-03,2021-11-05,59 days
2,"@christinebarnum #hokk is climbing the moon, d...",Twitter for Android,everyone,0,0,0,0,2022-01-03,2021-11-05,175,...,4764,False,photo,1478018721079861251-1456752050486841345-3_1478...,0,0,1,2022-01-03,2021-11-05,59 days
3,"@APompliano #hokk is climbing the moon, do not...",Twitter for Android,everyone,0,0,0,0,2022-01-03,2021-11-05,175,...,4764,False,photo,1478018549490896898-1456752050486841345-3_1478...,0,0,1,2022-01-03,2021-11-05,59 days
4,MULTI FR24 MISSED APPROACH ALERT : At Mon Jan ...,FlightAlerts,everyone,0,0,0,0,2022-01-03,2017-07-31,1855,...,1635215,False,photo,1478018535196663812-892059375426297858-3_14780...,1,1,1,2022-01-03,2017-07-31,1617 days


In [497]:
master_df.columns
hue_variables = ['verified','reply_settings', 'type','']

Index(['in_reply_to_user_id', 'referenced_tweets', 'text', 'lang', 'source',
       'reply_settings', 'entities', 'ret_count', 'reply_count', 'like_count',
       'quote_count', 'created_at', 'profile_created_at', 'profile_entities',
       'followers_count', 'following_count', 'user_tweet_count', 'username',
       'verified', 'type', 'id'],
      dtype='object')

In [516]:
# get hashtag flags from entities and drop
master_df['profile_entities'] = master_df['profile_entities'].astype(str)
master_df['entities'] = master_df['entities'].astype(str)

master_df['profile_ht'] = np.where(master_df['profile_entities'].str.contains('hashtags'),1,0)
master_df['tweet_ht'] = np.where(master_df['entities'].str.contains('hashtags'),1,0)

master_df.drop(['profile_entities','entities'], axis = 1, inplace = True)

In [533]:
import datetime
master_df['created_at'] = pd.to_datetime(master_df['created_at']).dt.date
master_df['profile_created_at'] = pd.to_datetime(master_df['profile_created_at']).dt.date

master_df['time_diff'] = master_df['created_at']  - master_df['profile_created_at']
master_df['time_diff'] = master_df['time_diff'].dt.days

In [ ]:
# see if all have replies on
reply_settings

In [ ]:
# created_at - profile_created_at variable

'1478018549490896898-1456752050486841345-3_1478004526951149575'

In [475]:
master_df['tweet_id'][0:2]

0    1478018848087592963
1    1478018765589729285
Name: tweet_id, dtype: int64

In [476]:
str(master_df['tweet_id']) + "-" + str(master_df['author_id'])

'0     1478018848087592963\n1     1478018765589729285\n2     1478018721079861251\n3     1478018549490896898\n4     1478018535196663812\n5     1478018502250401793\n6     1478018467668238337\n7     1478018454477230081\n8     1478018452241616899\n9     1478018413926748162\n10    1478018080357773312\n11    1478018014146666499\n12    1478017668548632580\n13    1478017620276244481\n14    1478017546662129667\n15    1478017330999398405\n16    1478016991701221377\n17    1478016965604225030\n18    1478016940882939904\n19    1478016683579219973\nName: tweet_id, dtype: int64-0     1391116725849137154\n1     1456752050486841345\n2     1456752050486841345\n3     1456752050486841345\n4      892059375426297858\n5     1456752050486841345\n6      937954630813679616\n7     1456752050486841345\n8     1412094250867585025\n9              3355694953\n10    1388140740568436737\n11              113118015\n12    1279964401408319488\n13    1425865486185627654\n14    1408799613382021132\n15     784476292930347008